# Hyperparameter Tuning for Clustering Algorithms

## Introduction: From Manual Exploration to Automated Search

In your previous work with clustering, you've manually explored parameters using loops to examine curves, conducted basic stability analysis, and used evaluation metrics like silhouette scores and Davies-Bouldin indices. While this manual approach builds intuition, it becomes impractical as the parameter space grows and computations become expensive.

**The key insight**: Hyperparameter tuning tools **automate the search process** so that we can focus on designing a good **evaluation function** rather than manually trying every possibility.

Different tools have different conventions for this evaluation function:

- In **scikit-learn**, we pass a **scoring function** that the search will **maximize**.
- In many other libraries — especially deep learning frameworks and general-purpose optimizers like Hyperopt, Optuna, or Bayesian optimization — we pass a **loss function** that the search will **minimize**.

A scoring function and a loss function are two sides of the same coin:
- **Score:** higher is better  
- **Loss:** lower is better  

If you have one, you can usually get the other just by multiplying by -1.

---

## Parameters vs. Hyperparameters in Clustering

Before diving into optimization, we should be clear about what exactly we are optimizing.  **Parameters** are learned by the underlying algorithm. These are not the direct target of optimization. Examples of parameters for clustering algorithms are:

- **K-Means**: centroids (cluster center coordinates)
- **Gaussian mixture model**: means and covariances
- **Hierarchical clustering**: linkage distances at each merge

Optimization focuses on **hyperparameters**, which control how the underlying algorithm finds parameters.  Examples of hyperparameters include:

- **K-Means**: Number of clusters (`k`)
- **DBSCAN**: `epsilon` and `min_samples`
- **Hierarchical clustering**: Linkage criterion in hierarchical clustering
- **UMAP**: Dimensionality reduction settings (`n_neighbors` and `min_dist`)

**Key takeaway**: Hyperparameter optimization algorithms let us search over _hyperparameters_ systematically instead of by hand; the _parameters_ are still determined by the underlying algorithm we are trying to optimize.  A _loss function_ (or _scoring function_) guides the search.

---

## The Challenge in Clustering: Designing the Evaluation Function

In supervised learning, defining a loss function is usually straightforward — we measure prediction error (e.g., RMSE, cross-entropy) and minimize it.  

In clustering, the challenge is: **How do we translate “good clustering” into a number we can optimize?**

Possible objectives include:
- **Coverage:** How many points are assigned to a cluster?
- **Coherence:** How similar are points within the same cluster?
- **Stability:** Do clusters stay consistent across different runs?
- **Interpretability:** Can a domain expert make sense of them?

The standard metrics you’ve seen before — inertia, silhouette score, Davies–Bouldin index — are **knowledge-lean**: they capture geometric structure but know nothing about your specific domain. They’re useful heuristics, not universal measures of quality.

---

## Example: Using the Elbow Method as an Evaluation Function

Let’s look at a classic problem: choosing `k` in K-Means.

### Why Minimizing Inertia Fails

Thought experiment:
- `k=1`: one cluster → high inertia
- `k=2`: lower inertia
- `k=3`: even lower
- `k=n`: every point is its own cluster → inertia = 0

If we **minimize inertia** directly, we end up with `k=n`, which is useless.  

Instead, the **elbow** represents a sweet spot between fewer clusters (parsimony) and tighter clusters (low inertia).

---

### Turning the Elbow into a Function We Can Optimize

We can capture the “elbow” by looking at the **second derivative** of inertia with respect to `k`.  
Intuition: at the elbow, the rate of inertia improvement slows down sharply — i.e., curvature is high.

In this tutorial, we will:
1. Implement a function to compute an “elbow score” for each `k` based on curvature.
2. Wrap that into a **custom scorer** for `sklearn`’s `GridSearchCV` (remember: in scikit-learn, scorers are maximized).
3. Compare our elbow-based choice of `k` with another metric: the silhouette score.

---

📌 **A note about this example:**  
In most optimization settings, the evaluation function is computed for each candidate parameter setting independently.  
Here, the elbow score depends on **multiple `k` values at once**, so we precompute inertias over a whole range.  
This means that for our toy example, `GridSearchCV` isn’t really doing much “search” — but it’s a useful illustration of how custom scorers fit into the API.


---

### Step 1 — Generate some example data

We’ll use `make_blobs` to create a synthetic dataset with four clusters. This gives us a “ground truth” number of clusters, but we won’t use it for tuning — it’s just to know what the real answer is.

In [ ]:
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer, silhouette_score

# Generate synthetic dataset
X, y_true = make_blobs(
    n_samples=300, 
    centers=4, 
    n_features=2, 
    random_state=42, 
    cluster_std=1.5
)

### Step 2 — Define a function to calculate the "elbow score"

The elbow score is based on the **second derivative** of inertia with respect to `k`.

- The **first derivative** tells us how quickly inertia is decreasing.
- The **second derivative** tells us how much that rate of decrease is slowing down — a large positive second derivative means we’ve hit the “bend” in the curve.

We’ll also smooth the inertia values with a Gaussian filter to reduce noise.

In [ ]:
def calculate_elbow_score(inertias):
    """
    Calculate elbow score using a smoothed second derivative.
    Higher scores indicate stronger "elbow" points.
    """
    # Smooth the curve to reduce noise
    smoothed = gaussian_filter1d(inertias, sigma=0.5)
    
    # First derivative (rate of change)
    first_deriv = np.gradient(smoothed)
    
    # Second derivative (curvature)
    second_deriv = np.gradient(first_deriv)
    
    return second_deriv

### Step 3 — Create a custom scorer for GridSearchCV

`GridSearchCV` in sklearn expects a **scoring function** with the signature:

~~~python
scorer(estimator, X, y=None) -> float
~~~

Since clustering is unsupervised, we won’t use y.

Our plan:

- Decide on a range of possible k values.
- Fit K-Means for each k and store the inertia.
- Use our calculate_elbow_score function to compute elbow scores for all tested k.
- Create a lookup dictionary so that, when GridSearchCV tries a given k, our scorer returns the corresponding elbow score.

In [ ]:
def make_elbow_scorer(k_range, X):
    """Create an elbow score function for GridSearchCV."""
    inertias = []
    for k in k_range:   
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(X)
        inertias.append(km.inertia_)
    
    elbow_scores = calculate_elbow_score(inertias)
    elbow_lookup = dict(zip(k_range, elbow_scores))
    
    def elbow_scorer(estimator, X_test, y=None):
        # GridSearchCV will fit the estimator with the given n_clusters;
        # we just look up the precomputed elbow score for that k.
        return elbow_lookup.get(estimator.n_clusters, 0.0)

    return elbow_scorer
    

### Step 4 — Use GridSearchCV to tune `k`

Normally, `GridSearchCV` splits the data into train/test folds.  
Since we’re not predicting labels here, we just want it to evaluate each `k` **once** on all the data.

We can do this by passing a **custom CV splitter** that uses the whole dataset for both “train” and “test” in each fold.

Here, we’ll search for `k` from 2 to 10.

In [ ]:
from sklearn.model_selection import GridSearchCV,  PredefinedSplit

# Range of k values to test
k_values = range(2, 11)
n = X.shape[0]
one_fold = [(np.arange(n), np.arange(n))] # We only need one fold here

# Create the grid search object
grid = GridSearchCV(
    estimator=KMeans(random_state=42, n_init=10),
    param_grid={'n_clusters': k_values},
    scoring=make_elbow_scorer(k_values, X),
    cv=one_fold,
    verbose=0
)

# Run the search
grid.fit(X)

print("Best k according to elbow score:", grid.best_params_['n_clusters'])
print("Best elbow score:", grid.best_score_)

### Step 5 — Compare with the silhouette score

The **silhouette score** is another metric for evaluating clustering quality.  
It measures how similar points are to their own cluster compared to other clusters.

We can repeat the same `GridSearchCV` process but use `silhouette_score` as the metric.

In [ ]:
silhouette_scorer = lambda est, X, y=None: silhouette_score(X, est.labels_)

grid_sil = GridSearchCV(
    estimator=KMeans(random_state=42, n_init=10),
    param_grid={'n_clusters': k_values},
    scoring=silhouette_scorer,
    cv=one_fold
)

grid_sil.fit(X)

print("Best k by silhouette:", grid_sil.best_params_['n_clusters'])
print("Best silhouette score:", grid_sil.best_score_)

---

### Step 6 — Summary

- **The elbow method** is a way to encode our intuition about where adding more clusters stops helping much.
- By turning it into a **custom scoring function**, we can plug it directly into `GridSearchCV` or `RandomizedSearchCV`.
- In real-world problems, you might combine multiple metrics (e.g., elbow score, silhouette score) or use domain knowledge to guide hyperparameter tuning. For instance, you could 

This is a simple but important lesson:  
> In unsupervised learning, *your scoring function is your definition of success*.  
> Automated search is only as good as the way you measure “good.”



---

## Example: Balancing Coverage and Coherence in DBSCAN Hyperparameter Tuning

In real-world clustering, not every algorithm assigns *every* point to a cluster.  
For example, **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) and its cousin HDBSCAN can mark points as “noise” if they don’t belong to any dense region.

This creates a unique **trade-off**:

- If `eps` (the neighborhood radius) is too small, many points are left as noise → **low coverage**.
- If `eps` is too large, clusters start to merge and lose shape → **low coherence**.

We often want to guide the algorithm’s parameters to strike the right balance between:
1. **Coverage** – the proportion of points assigned to a cluster.
2. **Coherence** – how well-separated the resulting clusters are (measured here with the silhouette score).

---

### Step 1 — Create a dataset with noise

We’ll make a synthetic dataset using `make_blobs`, then add random noise points to simulate a messier real-world scenario.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, make_scorer
from sklearn.model_selection import GridSearchCV

np.random.seed(42)

# Clustered points
X_clusters, y_clusters = make_blobs(
    n_samples=300, 
    centers=4, 
    cluster_std=0.60, 
    random_state=42
)

# Noise points (uniformly scattered)
X_noise = np.random.uniform(low=-8, high=8, size=(50, 2))

# Combine into one dataset
X = np.vstack([X_clusters, X_noise])

plt.scatter(X[:, 0], X[:, 1], s=30, alpha=0.6)
plt.title("Synthetic Data: Clusters + Noise")
plt.show()

---

### Step 2 — Define coverage and quality

**Coverage**: the fraction of points that are *not* labeled as noise by DBSCAN (`label != -1`).

**Quality**: the silhouette score for the clustered points only (silhouette measures separation and compactness).  
⚠️ *Note*: The silhouette score is biased toward convex clusters — in more complex shapes, you might choose a different metric.

In [ ]:
def coverage_score(labels):
    """Proportion of points assigned to any cluster (label != -1)."""
    return (labels != -1).mean()

def coherence_score(X, labels):
    """Silhouette score for non-noise points."""
    mask = labels != -1
    if mask.sum() < 2 or len(np.unique(labels[mask])) < 2:
        return -1  # Not enough data to compute
    return silhouette_score(X[mask], labels[mask])

---

### Step 3 — Combine coverage and coherence into a single score

We’ll define a simple **weighted average**:

\[
\text{score} = w \times \text{coverage} + (1-w) \times \text{coherence}
\]

- If `w` is high → we prioritize including more points (coverage).
- If `w` is low → we prioritize cluster quality (coherence).

In [ ]:
def make_dbscan_scorer(coverage_weight=0.5):
    """Return a sklearn-compatible scorer for DBSCAN."""
    def score(estimator, X, y=None):
        labels = estimator.fit_predict(X)
        cov = coverage_score(labels)
        coh = coherence_score(X, labels)
        if coh == -1:
            return -1
        return coverage_weight * cov + (1 - coverage_weight) * coh
    return score

---

### Step 4 — Search over DBSCAN parameters with GridSearchCV

We’ll search over:
- `eps`: neighborhood radius
- `min_samples`: minimum number of points in a dense region

Because this is unsupervised, we’ll use a dummy “CV” split so each parameter setting is evaluated on all the data once.

In [ ]:
eps_values = np.linspace(0.2, 1.5, 10)
min_samples_values = range(3, 10)

n = X.shape[0]
one_fold = [(np.arange(n), np.arange(n))] # We only need one fold here

grid = GridSearchCV(
    estimator=DBSCAN(),
    param_grid={'eps': eps_values, 'min_samples': min_samples_values},
    scoring=make_dbscan_scorer(coverage_weight=0.8),
    cv=one_fold,
    verbose=0
)

grid.fit(X)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)

---

### Step 5 — Visualize the parameter search space

We can reshape the results into a table so we can see how the score changes across `eps` and `min_samples`.

In [ ]:
results_df = pd.DataFrame(grid.cv_results_)
pivot_table = results_df.pivot(
    index='param_min_samples',
    columns='param_eps',
    values='mean_test_score'
)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, annot=True, fmt=".2f", cmap="viridis")
plt.title("DBSCAN Coverage–Coherence Score Across Parameter Space")
plt.xlabel("eps")
plt.ylabel("min_samples")
plt.show()

---

### Step 6 — Inspect the best clustering

Let’s fit DBSCAN with the best parameters and see the resulting clusters and noise points.

In [ ]:
best_dbscan = DBSCAN(**grid.best_params_)
labels = best_dbscan.fit_predict(X)

plt.figure(figsize=(8, 6))
unique_labels = np.unique(labels)
for label in unique_labels:
    mask = labels == label
    if label == -1:
        color = "k"
        plt.scatter(X[mask, 0], X[mask, 1], c=color, s=20, label="Noise", alpha=0.5)
    else:
        plt.scatter(X[mask, 0], X[mask, 1], s=30, label=f"Cluster {label}")
plt.legend()
plt.title("Best DBSCAN Clustering")
plt.show()

---

### Step 7 — Try different coverage weights

Right now, we’re giving equal weight to coverage and coherence (`coverage_weight=0.5`).  
What happens if we:
- Increase it to 0.8 (favoring coverage)?
- Decrease it to 0.2 (favoring quality)?

📌 **Your task**:  
- Change `coverage_weight` in `make_dbscan_scorer`.
- Rerun the grid search.
- Compare the best parameters and resulting cluster shapes.

---

**Key takeaway**:  
In clustering, *how you define the score* is as important as the search method.  
Coverage vs. coherence is just one example — in your own problems, you may include stability, interpretability, or domain-specific rules in the score.


## Understanding Search Algorithms: From Grid to Bayesian Optimization

Now that we understand the importance of loss function design, let's explore different algorithms for searching the hyperparameter space.

### Search Methods Comparison

**Grid Search**: Exhaustive but expensive
- **How it works**: Try every combination in a predefined grid
- **Pros**: Guaranteed to find the best point in the grid
- **Cons**: Exponential growth with dimensions; no learning from previous evaluations
- **When to use**: Small parameter spaces (≤3 dimensions), unlimited compute

**Random Search**: Often surprisingly effective
- **How it works**: Randomly sample points from the parameter space
- **Pros**: Often outperforms grid search with the same budget
- **Cons**: No learning from previous evaluations
- **Key insight**: Most hyperparameters don't matter much; random search finds the few that do

**Bayesian Optimization with Hyperopt**: Smart exploration and exploitation
- **How it works**: Build a probabilistic model of the objective function
- **Pros**: Learns from previous evaluations; balances exploration vs exploitation
- **Algorithm**: Tree-structured Parzen Estimator (TPE)
- **When to use**: Expensive evaluations, complex parameter spaces

### Hyperopt and TPE: What's Happening Under the Hood?

Hyperopt uses **Tree-structured Parzen Estimator (TPE)**, a Bayesian optimization algorithm. Here's the intuition:

1. **Maintain two models**:
   - l(x): Model of parameter values that led to GOOD results
   - g(x): Model of parameter values that led to BAD results

2. **Select next point** by maximizing l(x)/g(x)
   - High ratio = likely to be good, not yet explored
   - This balances exploitation (areas we think are good) with exploration (uncertain areas)

3. **Update models** after each evaluation
   - More data → better models → smarter parameter selection

**The key insight**: TPE doesn't just randomly search; it actively learns which regions of parameter space are promising.

---

## Example: Using Hyperopt with DBScan

Previously, we used **Grid Search** to explore the hyperparameter space for DBSCAN.  
Grid Search is easy to understand, but it doesn’t *learn* from past trials — every parameter setting is tested with no regard for what we’ve already learned.

Now we’ll look at a smarter approach: **Bayesian Optimization** with [Hyperopt](https://github.com/hyperopt/hyperopt), which uses the **Tree-structured Parzen Estimator (TPE)** algorithm.

---

### Step 1 — Recap: Loss vs. Score

Remember:

- **scikit-learn GridSearchCV** expects a **score** (maximize).
- **Hyperopt** expects a **loss** (minimize).

If you already have a score you want to maximize, just negate it before returning.

---

### Step 2 — DBSCAN Coverage + Coherence

We’ll reuse our **coverage** and **coherence** metrics from the previous section:

- **Coverage:** fraction of points assigned to a cluster (`label != -1`).
- **Coherence:** silhouette score of the non-noise points.

We’ll combine them into a weighted average to form our **score**:

$$
\text{score} = w \times \text{coverage} + (1-w) \times \text{coherence}
$$

We then turn that score into a **loss** for Hyperopt by negating it.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

np.random.seed(42)

# --- Dataset: clusters + noise ---
X_clusters, _ = make_blobs(
    n_samples=2000, centers=6, cluster_std=0.70, random_state=42
)
X_noise = np.random.uniform(low=-8, high=8, size=(50, 2))
X = np.vstack([X_clusters, X_noise])

# --- Coverage & coherence metrics ---
def coverage_score(labels):
    return (labels != -1).mean()

def coherence_score(X, labels):
    mask = labels != -1
    if mask.sum() < 2 or len(np.unique(labels[mask])) < 2:
        return -1
    return silhouette_score(X[mask], labels[mask])

---

### Step 3 — Defining the objective function for Hyperopt

Hyperopt will repeatedly call this function with a set of parameters to test.  

We:
1. Fit DBSCAN with those parameters.
2. Calculate coverage and coherence.
3. Combine them into our weighted score.
4. Convert to a **loss** by negating (because Hyperopt minimizes).
5. Return the loss and a status flag.

In [ ]:
def objective_dbscan(params, X, coverage_weight=0.5, min_coverage=0.5):
    eps = params['eps']
    min_samples = int(params['min_samples'])
    
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X)
    
    cov = coverage_score(labels)
    coh = coherence_score(X, labels)
    
    if coh == -1:
        score = -1
    else:
        score = coverage_weight * cov + (1 - coverage_weight) * coh
    
    # Optional: penalize low coverage
    if cov < min_coverage:
        penalty = (min_coverage - cov) * 2
        score -= penalty
    
    return {'loss': -score, 'status': STATUS_OK}

---

### Step 4 — Defining the search space

Hyperopt’s `hp` module lets us define:

- **hp.uniform**: continuous range
- **hp.quniform**: quantized range (integers)

We’ll search over:
- `eps`: 0.1 to 3.0
- `min_samples`: integer between 3 and 20

In [ ]:
space = {
    'eps': hp.uniform('eps', 0.1, 3.0),
    'min_samples': hp.quniform('min_samples', 3, 20, 1)
}

---

### Step 5 — Simulating “initial random trials”

Unlike some Bayesian optimization libraries, Hyperopt doesn’t have a built-in `init_points` argument.  
If we want to start with random exploration, we can:

- First run `fmin` with `algo=hyperopt.rand.suggest` for N iterations to gather random data.
- Then continue with `algo=tpe.suggest` starting from the same `Trials` object.

This is useful when:
- You have no prior knowledge of the space.
- You want TPE to start with a diverse set of examples.

In [ ]:
from hyperopt import rand

# Store results across both phases
trials = Trials()

# Phase 1: random search for 10 trials
fmin(
    fn=lambda p: objective_dbscan(p, X, coverage_weight=0.5, min_coverage=0.7),
    space=space,
    algo=rand.suggest,
    max_evals=10,
    trials=trials,
    verbose=True
)

# Phase 2: TPE optimization for 40 trials
fmin(
    fn=lambda p: objective_dbscan(p, X, coverage_weight=0.8, min_coverage=0.7),
    space=space,
    algo=tpe.suggest,
    max_evals=50,   # total trials = 10 random + 40 TPE
    trials=trials,
    verbose=True
)

---

## Step 6 — Inspecting results

We can pull the best parameters and plot the optimization history.

In [ ]:
best_params = trials.best_trial['result']
best_score = -min([t['result']['loss'] for t in trials.trials])

print(f"Best score: {best_score:.4f}")
print("Best parameters:", trials.best_trial['misc']['vals'])

# Plot optimization history
losses = [-t['result']['loss'] for t in trials.trials]
eps_vals = [t['misc']['vals']['eps'][0] for t in trials.trials]
min_samples_vals = [t['misc']['vals']['min_samples'][0] for t in trials.trials]

plt.figure(figsize=(12, 4))

# Loss over iterations
plt.subplot(1, 2, 1)
plt.plot(losses, marker='o')
plt.axvline(9.5, color='red', linestyle='--', label='Switch to TPE')
plt.title('Optimization Progress')
plt.xlabel('Iteration')
plt.ylabel('Coherence–Coverage Score')
plt.legend()

# Parameter exploration
plt.subplot(1, 2, 2)
plt.scatter(eps_vals, min_samples_vals, c=losses, cmap='viridis')
plt.colorbar(label='Score')
plt.xlabel('eps')
plt.ylabel('min_samples')
plt.title('Parameter Space Exploration')

plt.tight_layout()
plt.show()

---

## Step 7 — Key takeaways

- Hyperopt minimizes **loss**, so we negate scores when needed.
- The **TPE algorithm** learns from past trials, focusing the search on promising areas.
- We can simulate an “initial random exploration” phase by manually running `rand.suggest` before `tpe.suggest`.
- The definition of the objective function (loss) is crucial — change the coverage weight and you’ll change the optimizer’s priorities.

---

📌 **Your turn**:
- Change `coverage_weight` to 0.2 and rerun the search.
- Observe how the “best” parameters change and how the search history looks.

---

## Comprehensive Example: Semantic Text Clustering with UMAP + HDBSCAN + Bayesian Optimization

In this example, we’ll cluster **sentences** (not paragraphs) using semantic embeddings, optional dimensionality reduction, and a density‑based algorithm. We’ll demonstrate how to design a **loss function** that balances **coverage** (how many sentences get clustered) with **coherence** (how semantically tight clusters are). Then we’ll tune HDBSCAN with **Hyperopt** (TPE Bayesian optimization), including a short **random exploration** phase before TPE.

**Pipeline overview:**
1. Create several long paragraphs on different topics → **split into sentences**.
2. Embed sentences with a **SentenceTransformer**.
3. Precompute **three UMAP projections** (different `n_neighbors`/`min_dist`, always 2D). We let the optimizer **choose** the best projection rather than fully tuning UMAP (smaller search space, faster).
4. Cluster using **HDBSCAN**.
5. Define a **loss** that combines:
   - **Coverage** = fraction of sentences assigned to any cluster (label != −1)
   - **Coherence** = average **pairwise cosine similarity** inside each cluster (weighted by cluster size), computed from embeddings
6. Use **Hyperopt** with a short **random** phase, then **TPE** to find good parameters.

> This is conceptually similar to **BERTopic**: semantic embeddings → optional projection → density clustering, with a tunable “tightness vs. coverage” trade‑off. There are many valid ways to combine coverage and coherence; try alternatives (e.g., minimum coverage thresholds, unweighted averages) after the demo.

In [ ]:
# --- Imports ---
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

import umap
import hdbscan

from hyperopt import fmin, tpe, rand, hp, STATUS_OK, Trials

from scipy.spatial import ConvexHull

np.random.seed(42)

### Step 1 — Build paragraphs, then split to sentences

We’ll craft **four long paragraphs (≥20 sentences each)** across distinct topics. The goal is to generate some **semantically off‑topic sentences** that naturally become **noise** under the right clustering settings. After creating paragraphs, we **split into sentences** and keep the **topic label** for plotting later.

In [ ]:
# Long paragraphs (each ≥20 sentences). Keep them short sentences for reliable splitting.
tech_para = ("Machine learning systems learn patterns from data. "
             "Neural networks stack layers to extract features. "
             "Transformers use attention to relate tokens. "
             "Pretraining captures broad knowledge. "
             "Finetuning adapts models to tasks. "
             "Embedding spaces encode meaning. "
             "Vector search retrieves similar items. "
             "Inference speed matters in production. "
             "Batching improves throughput. "
             "Quantization reduces model size. "
             "Distillation transfers knowledge. "
             "Evaluation requires careful datasets. "
             "Metrics must reflect objectives. "
             "Biases can appear in data. "
             "Safety requires constraints. "
             "Monitoring detects drift. "
             "Retraining restores performance. "
             "Edge devices need efficient models. "
             "Cloud scaling handles traffic. "
             "Cost control guides deployment. "
             "Documentation supports teams. "
             "Experiment tracking preserves results.")

cooking_para = ("Pasta water should be salty like the sea. "
                "Fresh herbs brighten flavors. "
                "Aromatics build complexity early. "
                "Low heat coaxes sweetness from onions. "
                "Proper searing develops fond. "
                "Deglazing captures savory bits. "
                "Emulsions need gradual fat addition. "
                "Resting meat preserves juices. "
                "Bread dough benefits from patience. "
                "Gluten development adds structure. "
                "Proofing time changes crumb. "
                "Steam helps oven spring. "
                "Thermometers prevent guesswork. "
                "Acidity balances richness. "
                "Salt enhances perception. "
                "Textures should contrast. "
                "Plating influences appetite. "
                "Season every layer. "
                "Waste less by reusing scraps. "
                "Stocks concentrate flavor. "
                "Simple dishes demand technique. "
                "Practice grows intuition.")

sports_para = ("Endurance builds with consistent training. "
               "Recovery makes adaptation possible. "
               "Hydration supports performance. "
               "Footwork dictates positioning. "
               "Coaches design effective drills. "
               "Teamwork creates passing lanes. "
               "Defensive spacing reduces chances. "
               "Strength training prevents injury. "
               "Mobility enhances range of motion. "
               "Warmups prime the nervous system. "
               "Cooldowns aid circulation. "
               "Video analysis reveals tendencies. "
               "Strategy varies by opponent. "
               "Tempo changes unsettle defenses. "
               "Composure matters under pressure. "
               "Sleep improves reaction time. "
               "Nutrition fuels workloads. "
               "Periodization structures seasons. "
               "Mindset shapes execution. "
               "Fundamentals win close games. "
               "Practice quality beats volume. "
               "Confidence grows from preparation.")

finance_para = ("Diversification reduces idiosyncratic risk. "
                "Correlation patterns shift over cycles. "
                "Liquidity dries up in stress. "
                "Volatility clusters in regimes. "
                "Risk premia are time varying. "
                "Macro surprises move markets. "
                "Earnings anchor valuations. "
                "Discount rates reflect policy. "
                "Inflation reshapes cash flows. "
                "Term structure signals expectations. "
                "Credit spreads price default risk. "
                "Hedging trades basis risks. "
                "Leverage amplifies drawdowns. "
                "Diversified funding stabilizes firms. "
                "Capital buffers absorb losses. "
                "Position sizing controls exposure. "
                "Stop losses enforce discipline. "
                "Rebalancing locks in drift. "
                "Transaction costs erode alpha. "
                "Governance influences outcomes. "
                "Regulation alters incentives. "
                "Data quality drives decisions.")

def split_sentences(paragraph, topic):
    # Simple sentence split (works well for our short sentences)
    sentences = [s.strip() for s in re.split(r'(?<=[.!?])\s+', paragraph) if s.strip()]
    return pd.DataFrame({"text": sentences, "topic": topic})

df = pd.concat([
    split_sentences(tech_para, "tech"),
    split_sentences(cooking_para, "cooking"),
    split_sentences(sports_para, "sports"),
    split_sentences(finance_para, "finance"),
], ignore_index=True)

print(df.head(8))
print(f"\nTotal sentences: {len(df)}  | by topic: {df['topic'].value_counts().to_dict()}")

### Step 2 — Embed sentences

We’ll use a compact, widely used model: `all-MiniLM-L6-v2`. Replace with your preferred model if needed.

In [ ]:
def embed_texts(texts, model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(texts, show_progress_bar=False, normalize_embeddings=True)

embeddings = embed_texts(df["text"].tolist())
embeddings.shape

### Step 3 — Precompute three UMAP projections (2D)

We’ll create **three** fixed UMAP settings and precompute the 2D projections. The optimizer will **choose** among them.  
You *could* include UMAP’s parameters directly in the search, but that expands the search space and runtime.

In [ ]:
umap_configs = [
    {"n_neighbors": 5,  "min_dist": 0.05},
    {"n_neighbors": 10, "min_dist": 0.10},
    {"n_neighbors": 25, "min_dist": 0.25},
]

umap_projections = []
for cfg in umap_configs:
    reducer = umap.UMAP(
        n_neighbors=cfg["n_neighbors"],
        min_dist=cfg["min_dist"],
        n_components=2,
        random_state=42,
        metric="cosine",  # cosine often works well for sentence embeddings
    )
    umap_projections.append(reducer.fit_transform(embeddings))

[len(p) for p in umap_projections]

### Step 4 — Define coverage and semantic coherence

- **Coverage:** fraction of sentences assigned to a cluster (`label != -1`).
- **Coherence (semantic):** for each cluster, compute the **pairwise cosine similarity matrix** for its member embeddings, take the **upper triangle mean**, and aggregate to a **global score weighted by cluster size**.

> Why weighted? Larger clusters should contribute proportionally more to global coherence. Try an unweighted average as an alternative exercise.

In [ ]:
def coverage_score(labels):
    labels = np.asarray(labels)
    return (labels != -1).mean()

def cluster_coherence_from_embeddings(embeds, labels):
    """
    Return (global_coherence, per_cluster_dict).
    global_coherence is the size-weighted mean of per-cluster mean cosine similarity.
    """
    labels = np.asarray(labels)
    mask = labels != -1
    if mask.sum() < 2 or len(np.unique(labels[mask])) < 1:
        return -1.0, {}

    per_cluster = {}
    totals = 0
    weighted_sum = 0.0

    for c in np.unique(labels[mask]):
        idx = np.where(labels == c)[0]
        if len(idx) < 2:
            per_cluster[int(c)] = -1.0
            continue
        sims = cosine_similarity(embeds[idx], embeds[idx])
        # take strict upper triangle to avoid self-similarity
        ut = np.triu_indices_from(sims, k=1)
        vals = sims[ut]
        if len(vals) == 0:
            per_cluster[int(c)] = -1.0
            continue
        coh = float(np.mean(vals))
        per_cluster[int(c)] = coh
        weighted_sum += coh * len(idx)
        totals += len(idx)

    if totals == 0:
        return -1.0, per_cluster

    global_coh = weighted_sum / totals
    # Rescale to [0,1] for visual comparability (optional, cosine sim already ~[0,1])
    return float(global_coh), per_cluster

### Step 5 — Define the objective (loss) for Hyperopt

We combine **coverage** and **coherence** into a single **score**, then negate it to produce a **loss** (Hyperopt minimizes).  
We also:
- Include a **minimum coverage** soft constraint with a penalty.
- Allow Hyperopt to **choose among three UMAP projections** via a categorical index.
- Tune HDBSCAN’s `min_cluster_size` and `min_samples`.

**Score definition:**

$$
\text{score} = w \cdot \text{coverage} + (1-w) \cdot \text{coherence}
$$

In [ ]:
def make_objective(umap_projections, embeddings, coverage_weight=0.5, min_coverage=0.5):
    # Note: coherence is computed from the original embeddings (semantic space),
    # while clustering happens in 2D (chosen UMAP projection).
    def objective(params):
        umap_ix = int(params["umap_ix"])
        min_cluster_size = int(params["min_cluster_size"])
        min_samples = int(params["min_samples"])

        coords_2d = umap_projections[umap_ix]

        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            cluster_selection_epsilon=0.0,
            metric="euclidean"  # on UMAP space
        )
        labels = clusterer.fit_predict(coords_2d)

        cov = coverage_score(labels)
        coh, _ = cluster_coherence_from_embeddings(embeddings, labels)

        if coh < 0:
            score = -1.0
        else:
            score = coverage_weight * cov + (1 - coverage_weight) * coh

        # Soft penalty for not meeting coverage target
        if cov < min_coverage:
            score -= (min_coverage - cov) * 2.0

        return {"loss": -score, "status": STATUS_OK, "labels": labels, "umap_ix": umap_ix}
    return objective

## Step 6 — Hyperopt search (random → TPE)

We simulate an **initial random phase** (10 trials), then continue with **TPE** for smarter exploration.  

Search space:
- `umap_ix` ∈ {0,1,2} (choose one of the precomputed UMAPs)
- `min_cluster_size` ∈ [3, 20]
- `min_samples` ∈ [1, 10]

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

space = {
    "umap_ix": hp.choice("umap_ix", [0, 1, 2]),
    "min_cluster_size": hp.quniform("min_cluster_size", 3, 20, 1),
    "min_samples": hp.quniform("min_samples", 1, 10, 1),
}

coverage_weight = 0.5   # try different numbers later!
min_coverage   = 0.2

objective = make_objective(umap_projections, embeddings,
                           coverage_weight=coverage_weight,
                           min_coverage=min_coverage)

trials = Trials()

# Phase 1: random
fmin(fn=objective, space=space, algo=rand.suggest,
     max_evals=10, trials=trials, verbose=True)

# Phase 2: TPE
fmin(fn=objective, space=space, algo=tpe.suggest,
     max_evals=50, trials=trials, verbose=True)

# Extract best trial
best_trial = min(trials.trials, key=lambda t: t["result"]["loss"])
best_umap_ix = best_trial["result"]["umap_ix"]
best_labels  = best_trial["result"]["labels"]
best_coords  = umap_projections[best_umap_ix]

best_cov = coverage_score(best_labels)
best_coh, per_cluster = cluster_coherence_from_embeddings(embeddings, best_labels)
best_score = coverage_weight * best_cov + (1 - coverage_weight) * best_coh

print(f"Best UMAP index: {best_umap_ix}  | config: {umap_configs[best_umap_ix]}")
print(f"Coverage: {best_cov:.3f}  | Coherence: {best_coh:.3f}  | Combined score: {best_score:.3f}")

## Step 7 — Visualize

- **Color** encodes the **paragraph topic** (tech/cooking/sports/finance).
- **Alpha** distinguishes **noise** (low alpha) from clustered points (high alpha).
- **Optional**: draw **convex hulls** around each discovered cluster to make cluster boundaries visible (skip clusters with < 3 points).

> Using color for topic lets you see whether clusters align with semantic origin. Using alpha for noise keeps color free for topic identity.

In [ ]:
topic_to_color = {"tech": "#1f77b4", "cooking": "#2ca02c", "sports": "#ff7f0e", "finance": "#9467bd"}
colors = df["topic"].map(topic_to_color).values

def plot_clusters(coords, labels, colors, draw_hulls=True, title="UMAP + HDBSCAN (semantic topics shown)"):
    plt.figure(figsize=(10, 8))
    labels = np.asarray(labels)
    unique_labels = sorted([l for l in np.unique(labels) if l != -1])

    # Noise first (low alpha)
    noise_mask = labels == -1
    plt.scatter(coords[noise_mask, 0], coords[noise_mask, 1],
                c=colors[noise_mask], alpha=0.25, s=30, label="Noise")

    # Clusters (higher alpha)
    for c in unique_labels:
        mask = labels == c
        plt.scatter(coords[mask, 0], coords[mask, 1],
                    c=colors[mask], alpha=0.9, s=35, label=f"Cluster {c}")

        # Optional convex hull outline
        if draw_hulls and mask.sum() >= 3:
            try:
                hull = ConvexHull(coords[mask])
                hull_pts = coords[mask][hull.vertices]
                plt.plot(np.append(hull_pts[:,0], hull_pts[0,0]),
                         np.append(hull_pts[:,1], hull_pts[0,1]),
                         linestyle='-', linewidth=1.5, color='black', alpha=0.6)
            except Exception:
                pass

    plt.title(title)
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    # Custom legend: topic colors + noise entry
    from matplotlib.lines import Line2D
    legend_items = [Line2D([0],[0], marker='o', color='w', label=t,
                           markerfacecolor=topic_to_color[t], markersize=8)
                    for t in topic_to_color]
    legend_items.insert(0, Line2D([0],[0], marker='o', color='w', label='Noise',
                                  markerfacecolor='gray', markeredgecolor='gray', alpha=0.3, markersize=8))
    plt.legend(handles=legend_items, loc="best", frameon=True)
    plt.tight_layout()
    plt.show()

plot_clusters(best_coords, best_labels, colors, draw_hulls=True,
              title=f"Best projection index {best_umap_ix} — clusters outlined, noise faded")

### Step 8 — Extensions and exercises

- **Change the trade‑off**: set `coverage_weight` to 0.2 (favor coherence) or 0.8 (favor coverage) and rerun. How do coverage and coherence respond?
- **Different aggregation**: try *unweighted* average of cluster coherences, or a **minimum coverage threshold** without weighting.
- **Alternative metrics**: experiment with other intra‑cluster similarity measures (e.g., median instead of mean; exclude outlier pairs).
- **UMAP in the optimizer**: replace the fixed `umap_ix` choice with continuous `n_neighbors` and `min_dist` in the search space. This increases compute but may find better optima.
- **HDBSCAN options**: tune `cluster_selection_epsilon` or `cluster_selection_method`.
- **Compare to BERTopic**: BERTopic automates many of these steps; after this lab, inspect how its parameters map onto the same trade‑offs.

**Key idea:** In unsupervised tasks, the *definition of the objective* (loss/score) encodes what “good” means. Your optimization is only as good as that definition.


---

## Summary and Guidelines

### When to Use Each Approach

1. **Grid Search** — Small, low-dimensional spaces where you want exhaustive coverage.
2. **Random Search** — Medium spaces or limited compute; strong baseline that often beats grid for the same budget.
3. **Bayesian Optimization (Hyperopt/TPE)** — Larger/complex spaces or expensive evaluations; learns from previous trials.
4. **Multi-objective Search** — When you explicitly need to balance **coverage vs. coherence** (e.g., by weighted sums or simple constraints like minimum coverage).

---

### Key Considerations for Clustering Hyperparameter Tuning

1. **Define Clear Objectives** (what “good” means for *your* task)
   - **Cluster quality** (e.g., silhouette, Davies–Bouldin; be mindful of biases toward spherical clusters).
   - **Coverage** (fraction of points assigned to clusters).
   - **Interpretability & domain relevance** (are clusters useful to humans?).
   - **Stability** (do results persist across reruns, perturbations, or small data changes?).

2. **Handle Incomplete Clustering** (DBSCAN, HDBSCAN)
   - Set a **minimum coverage** requirement or add a **penalty** when coverage is low.
   - Treat “noise” points as a feature, not a bug (they can signal outliers or off-topic content).
   - Align your metric with intent: it’s fine to accept lower coverage if your priority is very tight, coherent clusters.

3. **Validate Thoroughly — Without Ground Truth**
   - **Bootstrap stability analysis (what it means here):**  
     Refit your pipeline multiple times under small changes (e.g., resample sentences with replacement; jitter embeddings slightly; change UMAP seeds; re-run HDBSCAN).  
     Compare runs using **permutation-invariant** measures (e.g., pairwise co-assignment matrices, Adjusted Rand Index / NMI on aligned labels).  
     *Goal:* stable clusters across perturbations.
   - **Cross-validation for clustering (in this context):**  
     Split data (train/hold-out). Fit clusters on train. Then **assign** hold-out points to clusters (e.g., nearest centroid in embedding space, `hdbscan.approximate_predict`, or a k-NN to cluster cores).  
     Evaluate hold-out **coherence** (e.g., mean in-cluster cosine similarity) and **coverage**.  
     *Goal:* clusters generalize beyond the data used to form them.
   - **Visual inspection with DR:**  
     Use UMAP/TSNE/PCAs for sanity checks—look for structure that matches metrics.
   - **Domain expert review:**  
     Short summaries/examples per cluster; assess usefulness and face validity.

4. **Scale (and Metric) Appropriately**
   - Distance choices matter: **cosine** is often better for text embeddings; Euclidean can be sensitive to scale.
   - If using Euclidean methods (e.g., k-means, Euclidean DBSCAN), apply **standardization**; for cosine, **L2-normalize** embeddings.
   - Verify scaling/metric choices with domain knowledge (e.g., cosine for sentences).

5. **Computational Efficiency**
   - Set **time/iteration budgets** and adopt **early stopping** (e.g., stop after N no-improvement trials).
   - **Cache** expensive steps (embeddings, UMAP projections, distance matrices).
   - **Parallelize** where possible (e.g., `n_jobs`, batched inference).
   - Start simple (smaller grids / fewer trials), then expand if needed.

---

### Common Pitfalls to Avoid

1. **Over-optimizing internal metrics** (e.g., silhouette favors convex clusters; don’t chase a single number).
2. **Ignoring domain knowledge** (a high score ≠ useful clusters).
3. **Skipping stability checks** (results should persist under small changes).
4. **Treating noise as failure** (noise can be informative).
5. **Using the wrong distance/scale** (mismatch between data geometry and metric undermines results).

The art of unsupervised tuning is balancing multiple objectives while staying grounded in domain goals. Start simple, **validate stability and generalization**, and always inspect results visually and conceptually.

---

## Practical Decision Framework: Which Search Method to Use?

### Decision Tree for Method Selection

Clustering Hyperparameter Tuning Decision Framework:

```text

1. How many hyperparameters?
   ├─ 1–2 → Manual exploration or Grid Search
   ├─ 3–4 → Random Search (Grid if budget allows)
   └─ 5+  → Random Search or Bayesian Optimization

2. How expensive is each evaluation?
   ├─ Fast (< 1s)      → Grid or Random
   ├─ Medium (1–60s)   → Random or Bayesian (TPE)
   └─ Slow (> 1 min)   → Bayesian (TPE), small budget

3. How much domain knowledge?
   ├─ High   → Manual exploration → local optimization
   ├─ Medium → Random with informed ranges
   └─ Low    → Bayesian to learn relationships

4. What’s the compute budget?
   ├─ Limited (< 50 evals)   → Bayesian (TPE)
   ├─ Moderate (50–500)      → Random
   └─ High (> 500, low-dim)  → Grid

```

### Example Scenarios

**Scenario 1: Quick K-means tuning**  
1 parameter (`k`), very fast.  
**Use:** Manual elbow or tiny grid.

**Scenario 2: DBSCAN on a moderate dataset**  
2 parameters (`eps`, `min_samples`), medium cost.  
**Use:** Random search or TPE (≈30–50 evals).

**Scenario 3: UMAP + HDBSCAN on large text**  
4+ parameters, slow (embeddings + DR).  
**Use:** TPE with a small, carefully chosen budget (e.g., 20–30 evals), cache embeddings/UMAP.

**Scenario 4: Real-time re-tuning**  
Frequent updates, tight latency.  
**Use:** Fast random search + early stopping; reuse caches.

---

## Final Thoughts: The Meta-Learning Perspective

As you practice, you’ll build intuition about:
1. **Parameter interactions** (e.g., how `eps` and `min_samples` trade off in DBSCAN).
2. **Data characteristics** (e.g., high dimensionality favors cosine; scale matters).
3. **Domain patterns** (typical ranges that work for *your* tasks).
4. **Evaluation trade-offs** (when coverage should trump coherence—and vice versa).

This becomes part of your **domain expertise**—the secret sauce that turns generic optimizers into problem-solving tools. Remember: perfection isn’t the target; **actionable, trustworthy clusters** are. A “pretty good” clustering that experts can use often beats a “perfect” one that they can’t interpret.